# Oil Wells Selection & Region Profit Analysis (OilyGiant)

## Carga y preparación de los datos

Los datos se limpiaron correctamente, sin valores nulos ni duplicados lo cual lo hacen adecuados para entrenar modelo.

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st

PRICE_PER_UNIT = 4500
TOTAL_BUDGET = 100_000_000

file_paths = ['/datasets/geo_data_0.csv', '/datasets/geo_data_1.csv', '/datasets/geo_data_2.csv']

def load_data(file_path):
    data = pd.read_csv(file_path)
    return data

## Entrenamiento y evaluacion del modelo

1. Se uso regresion lineal para predecir el volumen de reservas (product) por region.
2. Division del conjunto: 75% entrenamiento, 25% validacion.

Se obtuvieron los siguientes RMSE:
1. geo_data_0: 37.76
2. geo_data_1: 0.89
3. geo_data_2: 40.15

Se calcularon promedios de prediccion e intervalos de confianza.

In [21]:
#Entreno y evaluo el modelo
def train_and_evaluate(data):
    features = data.drop(columns=['id', 'product'])
    target = data['product']
    
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=42
    )
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    target_pred = model.predict(features_valid)
    
    rmse = mean_squared_error(target_valid, target_pred, squared=False)
    mean_product = target_pred.mean()
    conf_interval = st.t.interval(
        0.95, df=len(target_valid)-1, loc=np.mean(target_pred), scale=st.sem(target_pred)
    )

    return model, target_pred, target_valid, rmse, mean_product, conf_interval

## Preparacion para el calculo de ganancias

In [22]:
#Vamos a seleccionar los valores reales de los 200 pozos con mayor prediccion
def get_top_200_targets(target_pred, target_real):
    top_200_indices = target_pred.argsort()[-200:]
    return target_real.iloc[top_200_indices]

## Calculo de ganancias esperadas

Se eligieron los 200 pozos con mayor prediccion y se calcularon sus ganancias reales.

Ganancias estimadas:

• geo_data_0: $38.97M

• geo_data_1: $24.87M
    
• geo_data_2: $33.78M

In [23]:
#Calculamos las ganancias
def calculate_profit_from_real(target_real_top200, price_per_unit=4500, total_budget=100_000_000):
    total_revenue = target_real_top200.sum() * price_per_unit
    return total_revenue - total_budget

## Evaluacion de riesgos y ganancias (Bootstrapping)

Se aplico bootstrapping con 1000 simulaciones por region:

1. En cada iteracion se extrajeron 500 pozos aleatorios y se evaluo la ganancia real de los 200 mejores por prediccion.
   
2. Se calcularon los beneficios promedio, intervalos de confianza y el riesgo de perdida.


In [30]:
def calculate_profit(target_pred, wells_selected=200, price_per_unit=4500, total_budget=100_000_000):
    
    selected_reserves = np.sort(target_pred)[-wells_selected:]
    total_revenue = selected_reserves.sum() * price_per_unit
    avg_profit = total_revenue - total_budget
    return avg_profit

#Proceso de bootstrapping usando valores reales
def bootstrap_risk(target_valid, target_pred, n_samples=1000, sample_size=500, wells_selected=200):
    
    profits = []
    
#Unimos las predicciones y valores reales en un DataFrame
    df = pd.DataFrame({
        'target': target_valid,
        'prediction': target_pred
    }).reset_index(drop=True)
    
    for _ in range(n_samples):
#Realizo el muestreo con reemplazo de 500 pozos como indicaste
        sample = df.sample(n=sample_size, replace=True, random_state=None)
        
#Seleccione el top200 con por prediccion
        top_sample = sample.sort_values(by='prediction', ascending=False).head(wells_selected)
        
        total_reserve = top_sample['target'].sum()
        total_revenue = total_reserve * PRICE_PER_UNIT
        profit = total_revenue - TOTAL_BUDGET
        
        profits.append(profit)
    
    profits = np.array(profits)
    mean_profit = profits.mean()
    conf_interval = np.percentile(profits, [2.5, 97.5])
    risk = (profits < 0).mean() * 100
    
    return mean_profit, conf_interval, risk

In [33]:
#Evaluamos cada region
results = []

for file_path in file_paths:
    data = load_data(file_path)
    model, target_pred, target_valid, rmse, mean_product, conf_interval_pred = train_and_evaluate(data)
    
    
    avg_profit = calculate_profit(target_pred)
    mean_bootstrap_profit, conf_interval_bootstrap, risk = bootstrap_risk(target_valid, target_pred)

    results.append({
        'Archivo': file_path,
        'RMSE': rmse,
        'Promedio de reservas predicho': mean_product,
        'Intervalo de confianza de reservas': conf_interval_pred,
        'Ganancia esperada (real)': avg_profit,
        'Ganancia promedio bootstrapping': mean_bootstrap_profit,
        'Intervalo de confianza de ganancias': conf_interval_bootstrap,
        'Riesgo de perdidas (%)': risk
    })

In [34]:
#Mostramos los resultados
data_results = pd.DataFrame(results)
print(data_results)

                    Archivo       RMSE  Promedio de reservas predicho  \
0  /datasets/geo_data_0.csv  37.756600                      92.398800   
1  /datasets/geo_data_1.csv   0.890280                      68.712878   
2  /datasets/geo_data_2.csv  40.145872                      94.771024   

       Intervalo de confianza de reservas  Ganancia esperada (real)  \
0  (92.11098337245197, 92.68661644070339)              3.896658e+07   
1  (68.14330007695061, 69.28245600132463)              2.486938e+07   
2  (94.52399415145041, 95.01805360386837)              3.377982e+07   

   Ganancia promedio bootstrapping       Intervalo de confianza de ganancias  \
0                     4.037700e+06  [-1273697.2622262007, 9056058.292635318]   
1                     4.429336e+06    [659342.8325417899, 8581004.732780475]   
2                     3.813170e+06   [-1438289.5286008578, 8735198.30563424]   

   Riesgo de perdidas (%)  
0                     5.9  
1                     1.3  
2                

## Conclusion

Desoues de realizar el entrenamiento de modelos de regresion lineal, estimando ganancias reales y analizar el riesgo mediante bootstrapping, llegue a la conclusion que la region mas rentable es la region geo_data_0 porque ofrece la mayor ganancia real como tal esperada, pero la region geo_data_1 presenta el menor riesgo de pérdida (1.3%) y una ganancia promedio competitiva, por lo cual me lleva a brindar la recomendacion de seleccionar la region geo_data_1 ya que nos da el mejor equilibrio de rentabilidad y seguridad, la que la convierte en las mas confiable para realizar inversiones en la apertura de 200 pozos nuevos y de esta forma nuestro cliente reduce el riesgo a nivel financiero sin sacrificar la ganacias potencialmente significativas.
